# Imagined Ecology
## Jake Bauer, A15188631

### Setup

In [3]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
#reading training data
file = open('plant-data-final-encoded.txt')
text = file.read()

#reading names for generation
import csv
names = []
with open("the-plant-list.csv") as csvfile:
    reader = csv.reader(csvfile) 
    for row in reader: # each row is a list
        names.append(row)

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))
print(len(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))
    
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 1256

# Number of RNN units
rnn_units = 1024#512

if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')
    
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) 
#sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Length of text: 1302281 characters
81 unique characters
1302281
{
  '\n':   0,
  ' ' :   1,
  '"' :   2,
  '#' :   3,
  '%' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '+' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}
'\ufeff"\n\n\n\n\nPinus ' ---- characters mapped to int ---- > [80  2  0  0  0  0  0 41 60 65 72 70  1]
﻿
"






'\ufeff"\n\n\n\n\nPinus rigida\nPitch Pine, Northern Pitch Pine: \nHabitat\n    \n    \n      native to Eastern North'
' America\n      cold hardy to zone 4\n      often found on poor, sandy soil that fails to support other'
' species\n    \n    \n    \n    \n    Habit\n      and Form \n    \n      evergreen tree\n      open conical s'
"hape, becoming more irregular with age\n      typically 40 to 60' wide and 30 to 50' wide \n      mediu"
'm texture \n      medium growth rate, slows down as tree reaches mature height \n    \n    \n    \n      \n

### Training

In [4]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS=35
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Prediction shape:  (64, 100, 81)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.38514
Epoch 1/35
203/203 [==============================] - 22s 108ms/step - loss: 2.0450
Epoch 2/35
203/203 [==============================] - 19s 93ms/step - loss: 1.1473
Epoch 3/35
203/203 [==============================] - 20s 100ms/step - loss: 0.9340
Epoch 4/35
203/203 [==============================] - 18s 89ms/step - loss: 0.8412
Epoch 5/35
203/203 [==============================] - 19s 94ms/step - loss: 0.7831
Epoch 6/35
203/203 [==============================] - 19s 94ms/step - loss: 0.7392
Epoch 7/35
203/203 [==============================] - 20s 96ms/step - loss: 0.7027
Epoch 8/35
203/203 [==============================] - 21s 102ms/step - loss: 0.6716
Epoch 9/35
203/203 [==============================] - 20s 100ms/step - loss: 0.6439
Epoch 10/35
203/203 [==============================] - 19s 94ms/step - loss: 0.6166
Epoch 11/35
203/203 [==============================] - 19s 9

### Generate Text

In [5]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = .4

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

import random
for i in range(100):
    r = random.randint(1,23830)
    plant_name = names[r][2]
    plants = generate_text(model, start_string=plant_name);
    output_plants_text = open('100-generated-plants.txt','a');
    output_plants_text.write("\n\n--------------------------------------------------------------\n\n")
    output_plants_text.write(plants);
    #print(plants);